In [ ]:
import glob
import re

import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils import Graph

PATH = '/Users/davidc-local/Neutrinos/LArCADe/SimLArCADe/py/gas-vdrift/2PSI/v6/'

field_strength = np.array([30, 50, 100, 300, 500, 1000, 3000, 5000, 10000])#,50000,100000]) # V / cm
density = 1.15e19 # cm^-3
Td = 1e-17 # 1 Td = 10^-17 V * cm^2

reducedfield_v = (field_strength / density) / Td

In [ ]:
plt.rcParams.update({'font.size': 18})

In [ ]:
from scipy.optimize import curve_fit

def gauss(x,mu,sigma,A):
    norm = A/(np.sqrt(2*np.pi)*sigma)
    exp  = np.exp(-((x-mu)**2)/(2*sigma*sigma))
    return norm * exp
    
def FitGauss(vals,BINS=None):
    variance = np.std(vals)
    mean = np.mean(vals)
    if (BINS == None):
        BINS = np.linspace(mean-2*variance,mean+2*variance,11)
    binned,edges = np.histogram(vals,bins=BINS)
    bcenters = 0.5*(edges[1:]+edges[:-1])
    guess = [mean,variance/2.,np.max(binned)]
    try:
        popt,popv = curve_fit(gauss,bcenters,binned,p0=guess)
        pope = np.sqrt(np.diag(popv))
        return popt,pope,BINS,True# popt[0],popt[1],popt[2],1
    except:
        return mean,variance,BINS,False#,1,0
    #pope = np.sqrt(np.diag(popv))

In [ ]:
import scipy.interpolate as interpolate

def log_interp1d(xx, yy, kind='linear'):
    logx = np.log10(xx)
    logy = np.log10(yy)
    lin_interp = interpolate.interp1d(logx, logy, kind=kind)
    log_interp = lambda zz: np.power(10.0, lin_interp(np.log10(zz)))
    return log_interp

# gar drift velocity (needed to go from measurement to transverse diffusion coeff)
x = [1e-19, 3e-17, 2e-15] # E/n = 10^-17 V * cm^2 = 1 Td 
y = [9e4, 3e5, 2e7] # cm/sec

driftvel = log_interp1d(x, y)

In [ ]:
gardiff = open('lxcat/gar_diffusion.csv','r')
measurements_DL_v = []
EN_v = []
diff_v = []
for line in gardiff:
    words = line.split()
    #print (words)
    if (len(words) == 2): 
        EN_v.append(float(words[0]))
        diff_v.append(float(words[1]))
    if ( (len(EN_v) == len(diff_v)) and (len(EN_v) > 0) and (len(words) < 2) ):
        measurements_DL_v.append([EN_v,diff_v])
        EN_v = []
        diff_v = []

In [ ]:
gardiff = open('lxcat/gar_diffusion_DT.csv','r')
measurements_DT_v = []
EN_v = []
diff_v = []
for line in gardiff:
    words = line.split()
    #print (words)
    if (len(words) == 2):  
        EN = float(words[0]) # in Td
        ENMOD = float(EN * 1e-17)
        #print ('reduced field is %g'%ENMOD)
        #print ('Dt/mu is %g [0.1 Volts]'%float(words[1]))
        #print (ENMOD)
        EN_v.append(EN)
        driftvelocity = driftvel(ENMOD) # cm/sec
        #print ('drift velocity for En of %g is %g'%(ENMOD,driftvelocity))
        diff = (float(words[1]) / 10.) * driftvelocity * (1./ (ENMOD) ) * 100.
        #print ('diffusion [1/cm * 1/sec] = %g'%diff)
        diff_v.append(diff)
    if ( (len(EN_v) == len(diff_v)) and (len(EN_v) > 0) and (len(words) < 2) ):
        measurements_DT_v.append([EN_v,diff_v])
        EN_v = []
        diff_v = []

In [ ]:
fig = plt.figure(figsize=(6,6))

ctr = 0
for measurement in measurements_DT_v:
    if ctr == 0:
        plt.plot(measurement[0],measurement[1],'o--',color='c',label=r'$D_T$')
    else:
        plt.plot(measurement[0],measurement[1],'o--',color='c')
    ctr += 1


ctr = 0
for measurement in measurements_DL_v:
    if ctr == 0:
        plt.plot(measurement[0],measurement[1],'*--',color='orange',label=r'$D_L$')
    else:
        plt.plot(measurement[0],measurement[1],'*--',color='orange')
    ctr += 1
    
plt.xlabel('Reduced Electric Field E/N in Td')
plt.ylabel(r'Diffusion $\times$ gas density [meter $\times$ sec. ]${}^{-1}$',fontsize=16)
plt.xscale('log')
plt.legend()
plt.grid()
plt.show()

The below two cells load the necessary files and compute the diffusion coefficients. At various fields, $100$ electrons were spawned and allowed to propagate for $10$ ns.

In [ ]:
file_tree = {}

for field in field_strength:
    for file in glob.glob(f'{PATH}/{field}/*.txt'):
        key = int(re.search(r'(\d*)(?:V)', file).group(1))
        #print ('key is ',key)
        if (key in file_tree):
            file_tree[key].append(file)
        else:
            file_tree[key] = [file]

The below shows the estimation process of the diffusion coefficients: the variance of each coordinate of the electrons in the cloud is plotted as a function of time. The diffusion coefficients are the slopes of these lines.

In [ ]:
threshold = 3 # Throw out points more than n standard deviations above the mean

# in cm^2 / s
D_L_all = []
D_Ty_all = []
D_Tz_all = []

# diffusion coeff. times density in [m * s]^-1
field_L = []
DN_L_all = []
field_Ty = []
DN_Ty_all = []
field_Tz = []
DN_Tz_all = []

ctr = 0
for key, value in sorted(file_tree.items()):
    group = Graph(value)

    t_avg = np.mean([k[-1] for k in group.t])

    x_vals = np.array([k[-1] for k in group.x])
    y_vals = np.array([k[-1] for k in group.y])
    z_vals = np.array([k[-1] for k in group.z])

    popt,pope,BINS,success = FitGauss(x_vals)
    
    if (success):
        this_D_L = popt[1]
        D_L_all.append(this_D_L)
        m = (this_D_L * 1e-6)**2 / ( 2 * (t_avg * 1e-9) )
        DN_L_all.append( m * density * (1e6) )
        field_L.append(reducedfield_v[ctr])
    
    fig = plt.figure(figsize=(3,3))
    plt.hist(x_vals,bins=BINS,histtype='step',lw=2,color='b')
    plt.plot(BINS,gauss(BINS,*popt),'k--')
    plt.title('$D_L$ @ field of %i V/cm'%key)
    plt.show()
    
    popt,pope,BINS,success = FitGauss(y_vals)
    
    if (success):
        this_D_Ty = popt[1]
        D_Ty_all.append(this_D_Ty)
        m = (this_D_Ty * 1e-6)**2 / ( 2 * (t_avg * 1e-9) )
        DN_Ty_all.append( m * density * (1e6) )
        field_Ty.append(reducedfield_v[ctr])
    
    fig = plt.figure(figsize=(3,3))
    plt.hist(y_vals,bins=BINS,histtype='step',lw=2,color='b')
    plt.plot(BINS,gauss(BINS,*popt),'k--')
    plt.title('$D_Ty$ @ field of %i V/cm'%key)
    plt.show()    
    
    popt,pope,BINS,success = FitGauss(z_vals)
    
    if (success):
        this_D_Tz = popt[1]
        D_Tz_all.append(this_D_Tz)
        m = (this_D_Tz * 1e-6)**2 / ( 2 * (t_avg * 1e-9) )
        DN_Tz_all.append( m * density * (1e6) )
        field_Tz.append(reducedfield_v[ctr])
        
    fig = plt.figure(figsize=(3,3))
    plt.hist(z_vals,bins=BINS,histtype='step',lw=2,color='b')
    plt.plot(BINS,gauss(BINS,*popt),'k--')
    plt.title('$D_Tz$ @ field of %i V/cm'%key)
    plt.show() 
    #else:
    #    DN_Tz_all.append(0)
    ctr += 1

The cell below plots the diffusion coefficients as a function of bulk field strength. The data refers to https://lar.bnl.gov/properties/index.html#e-trans.

In [ ]:
# plt.rcParams['font.size'] = 14
# plt.rcParams['font.family'] = 'serif'
fig = plt.figure(figsize=(8, 6))


#plt.plot(field_strength, [4.8078, 5.2069, 5.7826, 6.3684, 6.8223, 6.9404, 6.5406, 5.8135, 5.2090, 4.4562, 3.9163, 3.5097, 3.1910, 2.9333, 2.7199], label='$D_L$ from [6]')
#plt.plot(field_strength, [4.8629, 5.5250, 6.9602, 9.4303, 13.1586, 17.2051, 19.1958, 18.1949, 16.5709, 14.2722, 12.5855, 11.3328, 10.3730, 9.6162, 9.0048], label='$D_T$ from [6]')


ctr = 0
for measurement in measurements_DT_v:
    if ctr == 0:
        plt.plot(measurement[0],measurement[1],'o--',color='c',label=r'$D_T$')
    else:
        plt.plot(measurement[0],measurement[1],'o--',color='c')
    ctr += 1


ctr = 0
for measurement in measurements_DL_v:
    if ctr == 0:
        plt.plot(measurement[0],measurement[1],'*--',color='orange',label=r'$D_L$')
    else:
        plt.plot(measurement[0],measurement[1],'*--',color='orange')
    ctr += 1

plt.plot(field_L , DN_L_all,'ro', label='$D_L$')
plt.plot(field_Ty, DN_Ty_all,'bo', label='$D_{T_y}$')
plt.plot(field_Tz, DN_Tz_all,'bo', label='$D_{T_z}$')
    
plt.xlabel('Reduced Electric Field E/N in Td')
plt.ylabel(r'Diffusion $\times$ gas density [m $\times$ s]${}^{-1}$')
plt.title('Diffusion in GAr')

plt.legend(loc=0)
plt.xscale('log')
plt.ylim([3e23,2e25])
plt.grid()
plt.tight_layout()
plt.savefig('plots/gar_diffusion.pdf', dpi=250)
plt.show()

# plt.savefig('lar_diffusion.pdf', bbox_inches='tight')

For our own purposes, we display scatter plots and histograms of snapshots of our simulated electrons.